In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import joblib
import requests
import pandas as pd
from datetime import datetime, timedelta
import json

In [2]:
# rf_model = joblib.load("../models/pm10/random_forest_model.pkl")
xgb_model = joblib.load("../models/pm10/xgboost_model.pkl")

In [3]:
x_scaler = joblib.load("../scalers/pm10/x_scaler.pkl")
y_scaler = joblib.load("../scalers/pm10/y_scaler.pkl")

In [4]:
latitude = 41.9981
longitude = 21.4254

url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m,relative_humidity_2m,windspeed_10m"
response = requests.get(url)

if response.status_code != 200:
    raise Exception("Error fetching weather data")

data = response.json()

hourly_data = {
    "time": data["hourly"]["time"],  
    "temperature": data["hourly"]["temperature_2m"],
    "humidity": data["hourly"]["relative_humidity_2m"],
    "wind_speed": data["hourly"]["windspeed_10m"]
}

df = pd.DataFrame(hourly_data)

df["time"] = pd.to_datetime(df["time"])

current_time = datetime.now()
next_2_days = current_time + timedelta(days=2)

df_next_2_days = df[(df["time"] >= current_time) & (df["time"] <= next_2_days)].copy()

df_next_2_days.loc[:, "hour"] = df_next_2_days["time"].dt.hour
df_next_2_days.loc[:, "year"] = df_next_2_days["time"].dt.year
df_next_2_days.loc[:, "month"] = df_next_2_days["time"].dt.month
df_next_2_days.loc[:, "day"] = df_next_2_days["time"].dt.day


X = df_next_2_days[['hour', 'year', 'month', 'day', 'temperature', 'humidity', 'wind_speed']]
X = x_scaler.transform(X)

# pm10_rf_predictions = rf_model.predict(X)
pm10_xgb_predictions = xgb_model.predict(X)


In [5]:
pm10_pred = y_scaler.inverse_transform(pm10_xgb_predictions.reshape(-1, 1))

In [6]:
pm10_pred.shape

(48, 1)

In [7]:
print(json.dumps({"pm10_predictions": pm10_pred.tolist()}))

{"pm10_predictions": [[36.35308074951172], [31.365257263183594], [31.478349685668945], [26.59033203125], [27.141992568969727], [26.45966911315918], [27.545452117919922], [25.422441482543945], [18.89374351501465], [34.55598831176758], [31.48291015625], [17.986909866333008], [27.826417922973633], [25.411880493164062], [24.60009765625], [21.372589111328125], [40.188480377197266], [59.926902770996094], [68.9573974609375], [76.02774047851562], [75.09040069580078], [77.10186004638672], [76.59227752685547], [59.24382019042969], [72.63277435302734], [60.45661926269531], [54.8821907043457], [53.426151275634766], [45.99031448364258], [47.41965103149414], [53.72421646118164], [55.34318923950195], [51.671775817871094], [49.63964080810547], [50.656124114990234], [51.566932678222656], [52.28364181518555], [45.526737213134766], [42.11295700073242], [38.593467712402344], [41.75626754760742], [61.877079010009766], [60.86857223510742], [61.183048248291016], [73.88203430175781], [72.12399291992188], [48.